In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data123/Volume II catalogue_removed.pdf


In [2]:
!apt-get update && apt-get install -y poppler-utils

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]                
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.9 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,229 kB] 
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]    
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,606 kB]        
Get:11 http://security.ubuntu.com/ubuntu jammy-securi

In [4]:
!pip install pdfplumber pytesseract pdf2image opencv-python


In [6]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"


In [11]:
!pip install -U langchain langchain-community langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 25.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [3]:
import pdfplumber
import pytesseract
import cv2
import numpy as np
from pdf2image import convert_from_path
import json
import pandas as pd
from IPython.display import FileLink

pdf_path = "/kaggle/input/data123/Volume II catalogue_removed.pdf"

def extract_text_from_pdf(pdf_path):
    text_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            # Extract text
            text = page.extract_text()
            if not text:
                # Convert image pages to text using OCR
                image = convert_from_path(pdf_path, dpi=300, first_page=i+1, last_page=i+1)[0]
                image = np.array(image)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                text = pytesseract.image_to_string(gray, lang="eng")
            
            text_data.append({"page": i+1, "content": text.strip()})
    return text_data

pdf_text_data = extract_text_from_pdf(pdf_path)
# 📌 Convert extracted text to DataFrame
df_text = pd.DataFrame(pdf_text_data)

# 📌 Save as CSV and JSON
output_csv = "/kaggle/working/extracted_text.csv"
output_json = "/kaggle/working/extracted_text.json"

df_text.to_csv(output_csv, index=False, encoding="utf-8")
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(pdf_text_data, f, ensure_ascii=False, indent=4)

# 📌 Generate Download Link
print("Download CSV:")
display(FileLink(output_csv))

print("\nDownload JSON:")
display(FileLink(output_json))


Download CSV:


/kaggle/working/extracted_text.csv


Download JSON:


/kaggle/working/extracted_text.json

In [8]:
!pip install tabula-py
!apt-get install -y default-jre  # Ensures Java is installed (required for Tabula)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 64.0 MB/s eta 0:00:00:00:01:01
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  default-jre-headless fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  default-jre default-jre-headless fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
0 upgraded, 10 newly installed, 0 to remove and 99 not upgraded.
Need to get 3,636 kB of archives.
After this operation, 12.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 default-jre-headless amd64 2:1.11-72build2 [3,042 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxtst6 amd64 2:1.2.3-1build4 [

In [9]:
import tabula
import pandas as pd

def extract_tables_from_pdf(pdf_path):
    """Extract tables from PDFs using Tabula."""
    tables = tabula.read_pdf(pdf_path, pages="all", multiple_tables=True)
    return tables

# Example usage
pdf_path = "/kaggle/input/data123/Volume II catalogue_removed.pdf"
pdf_tables = extract_tables_from_pdf(pdf_path)

# Save as CSV
for i, table in enumerate(pdf_tables):
    table.to_csv(f"/kaggle/working/table_{i+1}.csv", index=False)


In [1]:
from PIL import Image
from pdf2image import convert_from_path
import os
import shutil
from IPython.display import FileLink




# 📌 Ensure Poppler is installed (if needed)
poppler_path = "/usr/bin"

# 📌 Define output folder
output_image_folder = "/kaggle/working/extracted_images/"
os.makedirs(output_image_folder, exist_ok=True)

def extract_images_from_pdf(pdf_path, output_folder, poppler_path):
    """Extract images from PDF pages and save them as PNGs."""
    images = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)
    image_paths = []
    
    for i, img in enumerate(images):
        image_path = os.path.join(output_folder, f"page_{i+1}.png")
        img.save(image_path, "PNG")
        image_paths.append(image_path)
    
    return image_paths

# 📌 Run image extraction
pdf_path = "/kaggle/input/data123/Volume II catalogue_removed.pdf"
pdf_images = extract_images_from_pdf(pdf_path, output_image_folder, poppler_path)

# 📌 Create a ZIP file of extracted images
zip_file = "/kaggle/working/extracted_images.zip"
shutil.make_archive(zip_file.replace(".zip", ""), 'zip', output_image_folder)

print(f"✅ Extracted {len(pdf_images)} images. Download the ZIP file below.")

# 📌 Generate Download Link
display(FileLink(zip_file))


✅ Extracted 384 images. Download the ZIP file below.


/kaggle/working/extracted_images.zip

In [4]:
import spacy
from nltk.tokenize import sent_tokenize

# Load NLP Model for NER
nlp = spacy.load("en_core_web_sm")

pdf_path = "/kaggle/input/data123/Volume II catalogue_removed.pdf"
pdf_text_data = extract_text_from_pdf(pdf_path)

def structure_text(text_data):
    """Segment and categorize extracted text using NER."""
    structured_data = []
    
    for entry in text_data:
        doc = nlp(entry["content"])
        
        sections = []
        for sent in sent_tokenize(entry["content"]):
            # Categorize entities
            categories = {"ORG": [], "PERSON": [], "GPE": [], "DATE": [], "MONEY": []}
            for ent in nlp(sent).ents:
                if ent.label_ in categories:
                    categories[ent.label_].append(ent.text)
            
            sections.append({"sentence": sent, "entities": categories})
        
        structured_data.append({"page": entry["page"], "content": sections})
    
    return structured_data

structured_text_data = structure_text(pdf_text_data)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [6]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 52.9 MB/s eta 0:00:00:00:0100:01


In [7]:
!pip install faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.8 MB/s eta 0:00:00:00:0100:01


In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# 📌 Load Sentence Transformer Model for Embedding
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# 📌 Load Extracted Structured Text Data
structured_text_file = "/kaggle/working/extracted_text.json"

with open(structured_text_file, "r", encoding="utf-8") as f:
    structured_text_data = json.load(f)

def embed_text(data):
    """Converts structured text into embeddings using SentenceTransformer."""
    texts = []
    for entry in data:
        if isinstance(entry, dict) and "content" in entry:
            texts.append(entry["content"])  # Using full extracted content
    
    embeddings = embedding_model.encode(texts, convert_to_numpy=True)
    return texts, embeddings

# 📌 Convert Extracted Text to FAISS-Compatible Embeddings
texts, text_embeddings = embed_text(structured_text_data)

# 📌 Create & Store FAISS Index
dimension = text_embeddings.shape[1]
text_index = faiss.IndexFlatL2(dimension)
text_index.add(text_embeddings)

# 📌 Save FAISS Index to Disk
faiss_index_path = "/kaggle/working/text_faiss_index.bin"
faiss.write_index(text_index, faiss_index_path)

print(f"✅ FAISS index saved successfully: {faiss_index_path}")



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

✅ FAISS index saved successfully: /kaggle/working/text_faiss_index.bin


In [9]:
from torchvision import models
from torchvision.transforms import ToTensor
import torch

# Load ResNet for Image Embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(weights="IMAGENET1K_V1").to(device)
resnet.eval()

def extract_image_features(image_paths):
    """Convert images to feature vectors."""
    image_vectors = []
    transform = ToTensor()
    
    for img_path in image_paths:
        img = Image.open(img_path).convert("RGB").resize((224, 224))
        img_tensor = transform(img).unsqueeze(0).to(device)
        
        with torch.no_grad():
            features = resnet(img_tensor).squeeze().cpu().numpy()
        
        image_vectors.append(features)
    
    return np.array(image_vectors)

image_embeddings = extract_image_features(pdf_images)

# Store in FAISS
image_index = faiss.IndexFlatL2(image_embeddings.shape[1])
image_index.add(image_embeddings)
faiss.write_index(image_index, "image_faiss_index.bin")


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 176MB/s] 


In [13]:
!pip install -U langchain langchain-community langchain-openai langchain-huggingface


In [16]:
import os
import faiss
import json
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS as LCFAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import JSONLoader

# 📌 Define Paths
faiss_index_dir = "/kaggle/working/faiss_index"
os.makedirs(faiss_index_dir, exist_ok=True)

# =====================================================
# 📌 1️⃣ Save FAISS Index Correctly
# =====================================================
def save_faiss_index(embeddings, index_path):
    """Saves FAISS index properly in a folder"""
    dimension = embeddings.shape[1]
    faiss_index = faiss.IndexFlatL2(dimension)
    faiss_index.add(embeddings)

    faiss.write_index(faiss_index, os.path.join(index_path, "index.faiss"))
    print(f"✅ FAISS index saved at {index_path}/index.faiss")

# Load Sentence Transformer Model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Sample Text Data (Replace with actual structured text)
text_data = [
    {"content": "This is a sample document about AI."},
    {"content": "FAISS is useful for vector search."},
    {"content": "Multi-RAG improves retrieval-augmented generation."}
]

# Convert text to embeddings
texts = [entry["content"] for entry in text_data]
text_embeddings = embedding_model.encode(texts, convert_to_numpy=True)

# Save FAISS Index
save_faiss_index(text_embeddings, faiss_index_dir)

# =====================================================
# 📌 2️⃣ Load FAISS Correctly
# =====================================================
print("\n🔹 Loading FAISS Index...")

text_faiss = LCFAISS.load_local(
    faiss_index_dir,  # ✅ Load from directory, not a file
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    allow_dangerous_deserialization=True
)

# =====================================================
# 📌 3️⃣ Load Tables (JSON) & Set Up Multi-RAG
# =====================================================
table_json_path = "/kaggle/working/tables.json"  # Replace with actual table path
json_loader = JSONLoader(table_json_path)

# Multi-RAG Retriever
retrievers = [text_faiss.as_retriever(), json_loader]

# GPT-powered Q&A
qa = RetrievalQA.from_chain_type(llm=OpenAI(model_name="gpt-4"), retriever=retrievers)

query = "What are the key important details from the document?"
response = qa.run(query)

print("\n🔹 Response:", response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ FAISS index saved at /kaggle/working/faiss_index/index.faiss

🔹 Loading FAISS Index...


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/faiss_index/index.pkl'